In [4]:
!pip install lottie

     ------------------------------------ 151.9/151.9 kB 452.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for lottie: filename=lottie-0.7.0-py3-none-any.whl size=173539 sha256=3aefa4c76347bdfd2e284812e5e56c77cb26daaf6aba44207d605074c93fb326
  Stored in directory: c:\users\ssorawit\appdata\local\pip\cache\wheels\2f\59\89\18612eac07ac6afe32f262b4ee1d9bec790dfb482baf54d4ea
Successfully built lottie


In [7]:
import lottie

In [1]:
import pandas as pd
import numpy as np
# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
file_name = 'df_mu.csv' ######### change file name here

df = pd.read_csv(file_name)

df_message = df.copy()
df_message.rename(columns={'trans_eng':'messages_eng',}, inplace=True)
df_message.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196498 entries, 0 to 196497
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   timestamp     196498 non-null  object
 1   time          196498 non-null  object
 2   time_text     196498 non-null  object
 3   message       196498 non-null  object
 4   dtect_lang    196498 non-null  object
 5   messages_eng  196498 non-null  object
dtypes: object(6)
memory usage: 9.0+ MB


In [10]:
# convert emoji to text 
import emoji
def convert_emoji_to_text(text):
    return emoji.demojize(text)

df_message['message_emo'] = df_message['message'].apply(convert_emoji_to_text)
df_message.head()

,timestamp,time,time_text,message,dtect_lang,messages_eng,message_emo
0,2023-01-15 07:48:30,07:48:30,-0:04,🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷costa rica 🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨...,it,🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷Costa Rica 🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨...,:Costa_Rica::Costa_Rica::Costa_Rica::Costa_Ric...
1,2023-01-15 07:48:30,07:48:30,-0:04,Indonesian,en,Indonesian,Indonesian
2,2023-01-15 07:48:30,07:48:30,-0:04,🇨🇴,und,🇨🇴,:Colombia:
3,2023-01-15 07:48:30,07:48:30,-0:04,🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_...,en,🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_...,:Spain::red_heart::Spain::red_heart::Spain::re...
4,2023-01-15 07:48:31,07:48:31,-0:04,ASIAN BEAUTY QUEENS ARE HORRIBLE,en,ASIAN BEAUTY QUEENS ARE HORRIBLE,ASIAN BEAUTY QUEENS ARE HORRIBLE


In [12]:
###### 3.1 extract from name ######
import pycountry
def extract_country(text):
    for country in pycountry.countries:
        # if country.name or country.alpha_2 or country.alpha_3 or country.flag in text:
        if country.name.lower() in text.lower():
            return country.name   
    return 'N/A'

###### 3.2 country_nospace ######
def extract_country_nospace(text):
    for country in pycountry.countries:
        # if country.name or country.alpha_2 or country.alpha_3 or country.flag in text:
        if country.name.lower().replace(' ', '') in text.lower():
            return country.name   
    return 'N/A'

###### 3.3 flag ######
def extract_flag(text):
    for country in pycountry.countries:
        # if country.name or country.alpha_2 or country.alpha_3 or country.flag in text:
        if country.flag in text:
            return country.name        
    return 'N/A'


###### 3.5 regex ######
import re
import pycountry

def extract_country_re(text):
    # Get a list of all country names
    countries = [country.name.lower() for country in pycountry.countries]
    # Join the country names into a single string with '|' separators
    pattern = '|'.join(countries)
    # Compile the regular expression pattern
    regex = re.compile(pattern)
    # Find the first match in the text
    match = regex.search(text.lower())
    # Return the matched country name, or 'N/A' if no match was found
    return match.group() if match else 'N/A'

def extract_country_re2(text):
    # Get a list of all country names and common abbreviations
    countries = [country.name for country in pycountry.countries]
    countries += [country.alpha_2 for country in pycountry.countries]
    countries += [country.alpha_3 for country in pycountry.countries]   
    # Join the country names and abbreviations into a single string with '|' separators
    pattern = '|'.join(countries)  
    # Add regex pattern to match variations in country names
    # (e.g. 'United States' or 'United St.' or 'U.S.' or 'USA')
    # Allow for optional spaces between words
    pattern = r'\b(?:{})\b'.format(pattern)
    pattern = pattern.replace(' ', r'\s*') 
    # Compile the regular expression pattern
    regex = re.compile(pattern, re.IGNORECASE)
    # Find the first match in the text
    match = regex.search(text)
    # Return the matched country name, or 'N/A' if no match was found
    return match.group() if match else 'N/A'

def extract_country_re3(text):
    # Get a list of all country names and common abbreviations
    countries = [country.name.lower() for country in pycountry.countries]
    countries += [country.name.lower().replace(' ', '') for country in pycountry.countries]
    countries += [country.alpha_2.lower() for country in pycountry.countries]
    countries += [country.alpha_3.lower() for country in pycountry.countries]
    # Join the country names and abbreviations into a single string with '|' separators
    pattern = '|'.join(countries)

    # Add regex pattern to match variations in country names
    # (e.g. 'United States' or 'United St.' or 'U.S.' or 'USA')
    # Allow for optional spaces between words
    pattern = r'\b(?:{})\b'.format(pattern)
    pattern = pattern.replace(' ', r'\s*')
    # Compile the regular expression pattern
    regex = re.compile(pattern, re.IGNORECASE)
    # Find all matches in the text
    matches = regex.findall(text)
    # Return the list of matches, or an empty list if no matches were found
    return matches

###### 3.6 NLTK ######
import nltk
def extract_country_nltk(text):
    try:
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        # Part-of-speech tag the tokens
        pos_tags = nltk.pos_tag(tokens)
        # Perform named entity recognition
        entities = nltk.ne_chunk(pos_tags)
        # Extract the location entities from the named entity tree
        locations = [e[0] for e in entities if isinstance(e, nltk.tree.Tree) and e.label() == 'GPE']
        # # Return the list of locations, or an empty list if no locations were found
        return locations
    except:
        return "error"

def extract_country_nltk_2(text):
    for i in enumerate(text):
        ii = i[1][0].lower()
        for country in pycountry.countries:
            if ii in country.name.lower():
                return country.name
            elif ii in country.name.lower().replace(' ', ''):
                return country.name
    return 'N/A'

def extract_country_nltk_3(text):    
    # Get a list of all country names and common abbreviations
    countries = [country.name.lower() for country in pycountry.countries]
    countries += [country.name.lower().replace(' ', '') for country in pycountry.countries]
    countries += [country.alpha_2.lower() for country in pycountry.countries]
    countries += [country.alpha_3.lower() for country in pycountry.countries]
   
    # Join the country names and abbreviations into a single string with '|' separators
    pattern = '|'.join(countries) 
    # Add regex pattern to match variations in country names
    # (e.g. 'United States' or 'United St.' or 'U.S.' or 'USA')
    # Allow for optional spaces between words
    pattern = r'\b(?:{})\b'.format(pattern)
    pattern = pattern.replace(' ', r'\s*')
    for i in enumerate(text):
        ii = i[1][0].lower()
        if ii in pattern:
            return ii
    return 'N/A'

In [13]:
# Step 3.1: extract country from name
df_message['pycountry_name'] = df_message['messages_eng'].apply(extract_country)

In [14]:
# Step 3.2: extract country from country_nospace
df_message['pycountry_nospace'] = df_message['messages_eng'].apply(extract_country_nospace)

In [15]:
# Step 3.3: extract country from flag
df_message['pycountry_flag'] = df_message['message'].apply(extract_flag)

In [16]:
# Step 3.5: extract country from regex
df_message['country_re'] = df_message['messages_eng'].apply(extract_country_re)
df_message['country_re2'] = df_message['messages_eng'].apply(extract_country_re2)
df_message['country_re3'] = df_message['messages_eng'].apply(extract_country_re2)

In [17]:
# Step 3.6: extract country from nltk
df_message['country_nltk'] = df_message['messages_eng'].apply(extract_country_nltk)
df_message['country_nltk_2'] = df_message['country_nltk'].apply(extract_country_nltk_2)
df_message['country_nltk_3'] = df_message['country_nltk'].apply(extract_country_nltk_3)

In [18]:
df_message.columns

Index(['timestamp', 'time', 'time_text', 'message', 'dtect_lang',
       'messages_eng', 'message_emo', 'pycountry_name', 'pycountry_nospace',
       'pycountry_flag', 'country_re', 'country_re2', 'country_re3',
       'country_nltk', 'country_nltk_2', 'country_nltk_3'],
      dtype='object')

In [19]:
df_country = df_message[['timestamp', 'time', 'time_text', 'message', 'dtect_lang','messages_eng' ,'pycountry_name', 'pycountry_nospace', 'pycountry_flag', 'country_re', 'country_re2', 'country_re3', 'country_nltk_2', 'country_nltk_3']]
df_country.head()

,timestamp,time,time_text,message,dtect_lang,messages_eng,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
0,2023-01-15 07:48:30,07:48:30,-0:04,🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷costa rica 🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨...,it,🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷Costa Rica 🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨...,Costa Rica,N/A,Costa Rica,costa rica,Costa Rica,Costa Rica,N/A,N/A
1,2023-01-15 07:48:30,07:48:30,-0:04,Indonesian,en,Indonesian,Indonesia,Indonesia,N/A,indonesia,N/A,N/A,N/A,N/A
2,2023-01-15 07:48:30,07:48:30,-0:04,🇨🇴,und,🇨🇴,N/A,N/A,Colombia,N/A,N/A,N/A,N/A,N/A
3,2023-01-15 07:48:30,07:48:30,-0:04,🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_...,en,🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_...,N/A,N/A,Spain,N/A,N/A,N/A,N/A,N/A
4,2023-01-15 07:48:31,07:48:31,-0:04,ASIAN BEAUTY QUEENS ARE HORRIBLE,en,ASIAN BEAUTY QUEENS ARE HORRIBLE,N/A,N/A,N/A,N/A,ARE,ARE,N/A,N/A


In [20]:
# change to lower case
df_country['pycountry_name'] = df_country['pycountry_name'].str.lower()
df_country['pycountry_nospace'] = df_country['pycountry_nospace'].str.lower()
df_country['pycountry_flag'] = df_country['pycountry_flag'].str.lower()
df_country['country_re'] = df_country['country_re'].str.lower()
df_country['country_re2'] = df_country['country_re2'].str.lower()
df_country['country_re3'] = df_country['country_re3'].str.lower()
df_country['country_nltk_2'] = df_country['country_nltk_2'].str.lower()
df_country['country_nltk_3'] = df_country['country_nltk_3'].str.lower()

In [21]:
# vote for the country
df_country['country_vote'] = df_country[['pycountry_name', 'pycountry_nospace', 'pycountry_flag', 'country_re', 'country_re2', 'country_re3', 'country_nltk_2', 'country_nltk_3']].mode(axis=1)[0]
df_country

,timestamp,time,time_text,message,dtect_lang,messages_eng,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3,country_vote
0,2023-01-15 07:48:30,07:48:30,-0:04,🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷costa rica 🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨...,it,🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷Costa Rica 🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨🇷🇨...,costa rica,n/a,costa rica,costa rica,costa rica,costa rica,n/a,n/a,costa rica
1,2023-01-15 07:48:30,07:48:30,-0:04,Indonesian,en,Indonesian,indonesia,indonesia,n/a,indonesia,n/a,n/a,n/a,n/a,n/a
2,2023-01-15 07:48:30,07:48:30,-0:04,🇨🇴,und,🇨🇴,n/a,n/a,colombia,n/a,n/a,n/a,n/a,n/a,n/a
3,2023-01-15 07:48:30,07:48:30,-0:04,🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_...,en,🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_heart:🇪🇸:red_...,n/a,n/a,spain,n/a,n/a,n/a,n/a,n/a,n/a
4,2023-01-15 07:48:31,07:48:31,-0:04,ASIAN BEAUTY QUEENS ARE HORRIBLE,en,ASIAN BEAUTY QUEENS ARE HORRIBLE,n/a,n/a,n/a,n/a,are,are,n/a,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196493,2023-01-15 11:20:05,11:20:05,3:10:50,555555กลัวโดนทัวลงเเอน555,th,Hahahahaha,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a
196494,2023-01-15 11:20:11,11:20:11,3:10:56,take Thailand in what about his business :slig...,en,take Thailand in what about his business :slig...,thailand,thailand,n/a,thailand,thailand,thailand,thailand,thailand,thailand
196495,2023-01-15 11:20:11,11:20:11,3:10:56,USA ก็ไม่แย่นะ แต่แบบ แต่ก็นั่นแหละ อืมมม,th,"USA is not bad, but like that's it.",n/a,n/a,n/a,n/a,usa,usa,n/a,n/a,n/a
196496,2023-01-15 11:20:13,11:20:13,3:10:59,.,und,.,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a


In [23]:
# group by country
df_country_group = df_country.groupby(['country_vote']).count()
df_country_group

,timestamp,time,time_text,message,dtect_lang,messages_eng,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
country_vote,,,,,,,,,,,,,,
afghanistan,2,2,2,2,2,2,2,2,2,2,2,2,2,2
ai,1,1,1,1,1,1,1,1,1,1,1,1,1,1
albania,4,4,4,4,4,4,4,4,4,4,4,4,4,4
am,1,1,1,1,1,1,1,1,1,1,1,1,1,1
and,4,4,4,4,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
united states,41,41,41,41,41,41,41,41,41,41,41,41,41,41
vanuatu,1,1,1,1,1,1,1,1,1,1,1,1,1,1
viet nam,347,347,347,347,347,347,347,347,347,347,347,347,347,347


In [24]:
# view all countries
df_country_group.index.tolist()

['afghanistan',
 'ai',
 'albania',
 'am',
 'and',
 'angola',
 'are',
 'argentina',
 'armenia',
 'aruba',
 'as',
 'australia',
 'austria',
 'bahamas',
 'bahrain',
 'be',
 'belarus',
 'belgium',
 'belize',
 'bhutan',
 'brazil',
 'bulgaria',
 'burkina faso',
 'cambodia',
 'cameroon',
 'canada',
 'chad',
 'chile',
 'china',
 'colombia',
 'costa rica',
 'croatia',
 'cuba',
 'curaçao',
 'do',
 'dominica',
 'dominican republic',
 'ecuador',
 'egypt',
 'el salvador',
 'equatorial guinea',
 'es',
 'eth',
 'ethiopia',
 'finland',
 'france',
 'germany',
 'ghana',
 'greece',
 'greenland',
 'guatemala',
 'guinea',
 'haiti',
 'honduras',
 'hong kong',
 'hungary',
 'iceland',
 'im',
 'in',
 'ind',
 'india',
 'indonesia',
 'ireland',
 'is',
 'israel',
 'it',
 'italy',
 'jamaica',
 'japan',
 'jordan',
 'kyrgyzstan',
 'la',
 'lao',
 'lebanon',
 'malaysia',
 'mali',
 'malta',
 'mauritius',
 'me',
 'mexico',
 'mongolia',
 'morocco',
 'myanmar',
 'n/a',
 'namibia',
 'nauru',
 'nepal',
 'netherlands',
 'ni'

In [25]:
# drop not a country
df_country_group = df_country_group.drop(['n/a', 'ai', 'am', 'and', 'are','as','be','do','es','eth','im','in', 'ind', 'is', 'it','la', 'me', 'ni', 'sa'])
df_country_group

,timestamp,time,time_text,message,dtect_lang,messages_eng,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
country_vote,,,,,,,,,,,,,,
afghanistan,2,2,2,2,2,2,2,2,2,2,2,2,2,2
albania,4,4,4,4,4,4,4,4,4,4,4,4,4,4
angola,103,103,103,103,103,103,103,103,103,103,103,103,103,103
argentina,17,17,17,17,17,17,17,17,17,17,17,17,17,17
armenia,8,8,8,8,8,8,8,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
united states,41,41,41,41,41,41,41,41,41,41,41,41,41,41
vanuatu,1,1,1,1,1,1,1,1,1,1,1,1,1,1
viet nam,347,347,347,347,347,347,347,347,347,347,347,347,347,347


In [26]:
# change viet nam => vietnam
df_country_group = df_country_group.rename(index={'viet nam': 'vietnam'})
df_country_group

,timestamp,time,time_text,message,dtect_lang,messages_eng,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
country_vote,,,,,,,,,,,,,,
afghanistan,2,2,2,2,2,2,2,2,2,2,2,2,2,2
albania,4,4,4,4,4,4,4,4,4,4,4,4,4,4
angola,103,103,103,103,103,103,103,103,103,103,103,103,103,103
argentina,17,17,17,17,17,17,17,17,17,17,17,17,17,17
armenia,8,8,8,8,8,8,8,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
united states,41,41,41,41,41,41,41,41,41,41,41,41,41,41
vanuatu,1,1,1,1,1,1,1,1,1,1,1,1,1,1
vietnam,347,347,347,347,347,347,347,347,347,347,347,347,347,347


In [27]:
# group by country
df_country_group = df_country_group.groupby(['country_vote']).sum()
df_country_group

,timestamp,time,time_text,message,dtect_lang,messages_eng,pycountry_name,pycountry_nospace,pycountry_flag,country_re,country_re2,country_re3,country_nltk_2,country_nltk_3
country_vote,,,,,,,,,,,,,,
afghanistan,2,2,2,2,2,2,2,2,2,2,2,2,2,2
albania,4,4,4,4,4,4,4,4,4,4,4,4,4,4
angola,103,103,103,103,103,103,103,103,103,103,103,103,103,103
argentina,17,17,17,17,17,17,17,17,17,17,17,17,17,17
armenia,8,8,8,8,8,8,8,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
united kingdom,1,1,1,1,1,1,1,1,1,1,1,1,1,1
united states,41,41,41,41,41,41,41,41,41,41,41,41,41,41
vanuatu,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [28]:
df_country_group.index

Index(['afghanistan', 'albania', 'angola', 'argentina', 'armenia', 'aruba',
       'australia', 'austria', 'bahamas', 'bahrain',
       ...
       'thailand', 'timor-leste', 'trinidad and tobago', 'turkey', 'ukraine',
       'united kingdom', 'united states', 'vanuatu', 'vietnam', 'zimbabwe'],
      dtype='object', name='country_vote', length=105)

In [49]:
# summary of country and sort by number of messages
df_country_summary = df_country_group[['message']].sort_values(by=['message'], ascending=False)
# change column name
df_country_summary = df_country_summary.rename(columns={'message': 'amount'}).reset_index()
df_country_summary


,country_vote,amount
0,thailand,4802
1,philippines,3499
2,indonesia,3318
3,dominica,3204
4,nicaragua,3166
...,...,...
100,qatar,1
101,puertorico,1
102,mongolia,1
103,morocco,1


In [50]:
df_country_summary.to_csv('country_summary_mu.csv', index=False)

In [51]:
# replace Hongkong => Hong Kong
df_country_summary = df_country_summary.replace('hongkong', 'Hong Kong')
df_country_summary = df_country_summary.replace('lao', 'Laos')
df_country_summary = df_country_summary.replace('curaçao', 'Curacao')
df_country_summary = df_country_summary.replace('czechia', 'Czech Republic')
df_country_summary = df_country_summary.replace('timor-leste', 'East Timor')
df_country_summary = df_country_summary.replace('russian federation', 'Russia')
df_country_summary = df_country_summary.replace('puertorico', 'Puerto Rico')

In [32]:
df_country_group.index

Index(['afghanistan', 'albania', 'angola', 'argentina', 'armenia', 'aruba',
       'australia', 'austria', 'bahamas', 'bahrain',
       ...
       'thailand', 'timor-leste', 'trinidad and tobago', 'turkey', 'ukraine',
       'united kingdom', 'united states', 'vanuatu', 'vietnam', 'zimbabwe'],
      dtype='object', name='country_vote', length=105)

In [52]:
# Import the AWOC package.
import awoc
# Initialize the AWOC class.
my_world = awoc.AWOC()

# get countinent from country
df_country_summary['continent'] = df_country_summary['country_vote'].apply(lambda x: my_world.get_country_continent_name(x))
df_country_summary

,country_vote,amount,continent
0,thailand,4802,Asia
1,philippines,3499,Asia
2,indonesia,3318,Asia
3,dominica,3204,North America
4,nicaragua,3166,North America
...,...,...,...
100,qatar,1,Asia
101,Puerto Rico,1,North America
102,mongolia,1,Asia
103,morocco,1,Africa


In [53]:
# rename country_vote to country
df_country_summary = df_country_summary.rename(columns={'country_vote': 'country'})

# change to Upper case
df_country_summary['continent'] = df_country_summary['continent'].str.upper()
df_country_summary['country'] = df_country_summary['country'].str.upper()
df_country_summary

,country,amount,continent
0,THAILAND,4802,ASIA
1,PHILIPPINES,3499,ASIA
2,INDONESIA,3318,ASIA
3,DOMINICA,3204,NORTH AMERICA
4,NICARAGUA,3166,NORTH AMERICA
...,...,...,...
100,QATAR,1,ASIA
101,PUERTO RICO,1,NORTH AMERICA
102,MONGOLIA,1,ASIA
103,MOROCCO,1,AFRICA


In [55]:
df_country_summary.to_csv('country_summary_final_mu.csv', index=False)

# concat

In [56]:
df_mgi = pd.read_csv('country_summary_final.csv')
df_mu = pd.read_csv('country_summary_final_mu.csv')

In [59]:
df_mgi['owner'] = 'MGI'
df_mu['owner'] = 'MU'

In [62]:
df_mgi_mu_country = pd.concat([df_mgi, df_mu])
df_mgi_mu_country

,country,amount,continent,owner
0,THAILAND,12476,ASIA,MGI
1,CAMBODIA,5266,ASIA,MGI
2,BRAZIL,4360,SOUTH AMERICA,MGI
3,INDONESIA,3723,ASIA,MGI
4,PHILIPPINES,1807,ASIA,MGI
...,...,...,...,...
100,QATAR,1,ASIA,MU
101,PUERTO RICO,1,NORTH AMERICA,MU
102,MONGOLIA,1,ASIA,MU
103,MOROCCO,1,AFRICA,MU


In [63]:
# to csv
df_mgi_mu_country.to_csv('country_summary_final_mgi_mu.csv', index=False)
